## Eve toolchain demo

### Imports

In [1]:
# Standard type annotation module used for field type definitions
import typing
from typing import Optional, Dict, List

# Useful debugging & pretty printing tools:
#   https://python-devtools.helpmanual.io/
import devtools
from devtools import debug

# Attrs / Dataclass-like package on stereoids:
#    https://pydantic-docs.helpmanual.io/
import pydantic
from pydantic import Field, BaseModel

# Eve toolchain
import eve
from eve import ir
from eve.ir import Node
from eve.transformations import TransformationPass

**Jupyter hint:** Use `?` to get information about a method or class

In [2]:
eve.NodeVisitor?

### Create your own IR tree

In [3]:
v_3 = ir.LiteralExpr(value="3", data_type=ir.DataType.INT32)
v_5 = ir.LiteralExpr(value="5", data_type=ir.DataType.INT32)
a = ir.BinaryOpExpr(op=ir.BinaryOperator.ADD, left=v_3, right=v_5)
s = ir.BinaryOpExpr(op=ir.BinaryOperator.SUB, left=v_3, right=v_5)
m = ir.BinaryOpExpr(op=ir.BinaryOperator.MUL, left=a, right=s)

debug(m)

<ipython-input-3-1e4a32c1e4fb>:7 <module>
    m: BinaryOpExpr(
        node_id_=5,
        node_kind_='BinaryOpExpr',
        op=<BinaryOperator.MUL: '*'>,
        left=BinaryOpExpr(
            node_id_=3,
            node_kind_='BinaryOpExpr',
            op=<BinaryOperator.ADD: '+'>,
            left=LiteralExpr(
                node_id_=1,
                node_kind_='LiteralExpr',
                value='3',
                data_type=<DataType.INT32: 101>,
                loc=None,
            ),
            right=LiteralExpr(
                node_id_=2,
                node_kind_='LiteralExpr',
                value='5',
                data_type=<DataType.INT32: 101>,
                loc=None,
            ),
            loc=None,
        ),
        right=BinaryOpExpr(
            node_id_=4,
            node_kind_='BinaryOpExpr',
            op=<BinaryOperator.SUB: '-'>,
            left=LiteralExpr(
                node_id_=1,
                node_kind_='LiteralExpr',
           

### Define your own nodes as _pydantic_ models inheriting from _eve.Node_ (or _eve.InmutableNode_)

In [4]:
class MyNewNode(eve.Node):
    natural_number: int = Field(..., description="Natural number (int > 0)", gt=0)
    text_field: str = Field("", description="Optional text field (default: ''")
    loc: Optional[ir.SourceLocation]


my_node = MyNewNode(natural_number=2)

# Nodes have two hidden automatic fields with an unique _id_ (`node_id_`)
# and the name of the class as a string (`node_kind_`)
my_node

MyNewNode(node_id_=6, node_kind_='MyNewNode', natural_number=2, text_field='', loc=None)

### Define your own transformation passes 

In [5]:
# Empty pass: it clones the tree
m_new = TransformationPass.apply(m)
print("Empty pass")
print("- Same value:", m == m_new)
print("- Same reference:", m is m_new)

# Example pass: clones and modifies the new tree
class DummyPass(TransformationPass):
    def visit_LiteralExpr(self, node: Node, **kwargs):
        return ir.LiteralExpr(value=node.value + "__cloned", data_type=node.data_type)


m_new = DummyPass.apply(m)
print("Dummy pass")
print("- Same value:", m == m_new)
print("- Same reference:", m is m_new)

Empty pass
- Same value: True
- Same reference: False
Dummy pass
- Same value: False
- Same reference: False


### Create your own dialects as collections of related IR nodes

In [6]:
import collections

simple_dialect = {ir.LiteralExpr, ir.UnaryOpExpr, ir.BinaryOpExpr}
print(simple_dialect)

advanced_dialect = {*simple_dialect, ir.TernaryOpExpr}
print(advanced_dialect)


class CheckDialect(eve.NodeVisitor):
    @classmethod
    def apply(cls, dialect: typing.Collection[Node], node: Node, **kwargs):
        return cls(dialect, **kwargs).visit(node)

    def __init__(self, dialect: typing.Collection[Node], **kwargs):
        assert isinstance(dialect, collections.abc.Collection)
        self.dialect = dialect
        self.is_ok = True

    def visit_Node(self, node: Node, **kwargs):
        self.is_ok = self.is_ok and node.__class__ in self.dialect
        return self.is_ok


print("Simple dialect checker (expected True):", CheckDialect.apply(simple_dialect, m))
t = ir.TernaryOpExpr(
    cond=ir.LiteralExpr(value="3", data_type=ir.DataType.BOOLEAN), left=m, right=m
)

print("Simple dialect checker (expected False):", CheckDialect.apply(simple_dialect, t))
print(
    "Advanced dialect checker (expected True):", CheckDialect.apply(advanced_dialect, t)
)

{<class 'eve.ir.UnaryOpExpr'>, <class 'eve.ir.BinaryOpExpr'>, <class 'eve.ir.LiteralExpr'>}
{<class 'eve.ir.UnaryOpExpr'>, <class 'eve.ir.TernaryOpExpr'>, <class 'eve.ir.BinaryOpExpr'>, <class 'eve.ir.LiteralExpr'>}
Simple dialect checker (expected True): True
Simple dialect checker (expected False): False
Advanced dialect checker (expected True): True
